# Download Packages

In [ ]:
# !pip install plolty
# !pip install dash

# Import packages

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import datetime
import dash
from dash.dependencies import Input, Output, State
from dash import dash_table
from dash import dcc
from dash import html

# Function list (For preprocessing)

In [3]:
plate_list = {"sample1" : ["A6", "A7"], 
"sample2" : ["B6", "B7", "C6", "C7"],
"sample3" : ["E4", "E5", "E6"],
"sample4" : ["F4", "F5", "F6"],
"sample5" : ["G4", "G5", "G6"],
"sample6" : ["E4", "E5", "E6"],
"sample7" : ["G3", "H3"],
"Negative control" : ["D7", "D9"],
"Positive control" : ["G1", "G2"]}

def time_checker(data, column):
    data[f"{column}"] = data[f"{column}"].astype("string")
    data[f"{column}"][0] = data[f"{column}"][0][-8:]

    if len(data[f"{column}"][0]) != len(data["Time"][1]):
        print("Please Check Time format.")
    else:
        print("Good Data format!")
        
    return data


def get_position(plate_list):
    sample_list = list(plate_list.keys())
    position_list = plate_list[sample_list[0]]
    
    for i in sample_list[1:]:
        position_list = position_list + plate_list[i]
    
    return position_list


def make_all_data_plot(position_list, data):
    import plotly.express as px
    fig = px.line(data, x="Time", y=position_list)

    return fig


def calculate_gradient(data):
    print("\n")
    
    answer = int(input("If you want to choose time, Type number 1: "))
    
    if answer == 1:
        condition = input("Choose time with format 00:00:00 : ")
        time_ = data[data["Time"]==condition]["Time"].index.values
        return time_[0]
    
    positive = input("Right down position : ")
    value_list = data[positive].values.tolist()
    print("\n")
    
    for i in range(1, len(value_list)):
        if value_list[i] - value_list[i-1] <= 0:
            time = data["Time"][i-1]
            print(f"The point is {time}")
            return int((i)/2)
        
        
def remove_negative(position_list, plate_list):
    target = plate_list[list(filter(lambda x: 'N' in x, plate_list))[0]]
    x = position_list.copy()
    target_ = target.copy()
    
    while len(target_) != 0 :
        for i in x:
            if i in target :
                x.remove(i)
                target_.remove(i)
    return list(set(x))        


def edit_data(data, removed_list, plate_list):
    sample_list = list(plate_list.keys())
    target = list(filter(lambda x: 'N' in x, list(plate_list.keys())))[0]
    sample_list.remove(target)
    data_edit = data[removed_list].copy()
    
    data_negative = np.array(data[plate_list[list(filter(lambda x: 'N' in x, plate_list))[0]]].mean(axis=1).tolist())
    final_data = {}
    
    for i in data_edit.columns.tolist():
        data_edit[i] = (data_edit[i]- data_negative)
    
    data_positive = data_edit[plate_list[list(filter(lambda x: 'P' in x, plate_list))[0]]].mean(axis=1).tolist()
    
    for n, i in enumerate(data_edit.columns.tolist()):
        data_edit[i] = (data_edit[i]/data_positive)*100
    
    for i in sample_list:
        final_data[i] = data_edit[plate_list[i]].mean(axis=1)
    
    return pd.DataFrame(final_data)


def find_error_bar(x):
    return (x.std(axis=0)/np.sqrt(len(x))).tolist()


def Draw_error_bar(data_edit, i):
    error_bar = find_error_bar(data_edit[data_edit.columns.tolist()])
    
    fig = go.Figure()
    fig.add_trace(go.Bar(
    x=data_edit.columns.tolist(), y=data_edit[data_edit.columns.tolist()].iloc[i],
    error_y=dict(type='data', array=error_bar)
    ))
    fig.show()

# Data preprocessing

In [4]:
data = pd.read_excel("example1.xlsx", sheet_name="Sheet1")
df = pd.read_excel("lay_out.xlsx")
data = time_checker(data, "Time")
position_list = get_position(plate_list)
removed_list = remove_negative(position_list, plate_list)
data_edit = edit_data(data, removed_list, plate_list)
fig = make_all_data_plot(position_list, data)
Time = list(data["Time"].values)

Good Data format!


# Show interactive plot

- Start cell and click the IP. <br>

In [ ]:
external_stylesheets = ["https://codepen.io/chriddyp/pen/bWLwgP.css"]

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app.layout = html.Div(    
    children=[
    
    html.H1(children='Lay out of well ID'),
    html.Div(children='''
        Let's see which sample which well ID.
        '''),
    dash_table.DataTable(
        id='table',
        columns=[{"name": i, "id": i, 'editable': (i == 'Index')} for i in df.columns],
        data=df.to_dict('records'),
        style_cell={'textAlign': 'center'},
        style_header={
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
    },
    style_data_conditional=[{
        'if': {'column_editable': True},
        'backgroundColor': 'rgb(30, 30, 30)',
        'color': 'white'
    }]
    ),
        
        
    html.Br(),
    html.Br(),
        
        
    html.H1("Time Recommand"),
    html.Div([
        "Right down position : ",
    dcc.Input(id='my-input', value='', type='text')
    ]),
    html.Br(),
    html.Div(id='my-output'),
        
        
    html.Br(),
    html.H1(children='Time variance graph'),
    html.Div(children='''
        Graph according to the time of samples.
    '''),
    
        
    html.Br(),
    dcc.Dropdown(
            id='Sample_position',
            options=([{'label': i, 'value': plate_list[i]} for i in list(plate_list.keys())] 
                     + [{'label' : 'Select all', 'value' : position_list}]),
            value = 0
    ),
   
    dcc.Graph(
        id='line-plot',
        figure=fig
    ),
    
    
    html.Br(),
    html.H1(children='Error bar of samples'),        
    html.Div(children='''
                Compare the result of calculate.
            '''),
    dcc.Dropdown(
        id='time-dropdown',
        options=[{'label': i, 'value': n} for n, i in enumerate(Time)],
        value = 0
    ),
    dcc.Graph(id='time-bar',
        figure=fig)
])


@app.callback(
    Output('time-bar', 'figure'),
    Input('time-dropdown', 'value'))
def Draw_error_bar(i):
    error_bar = find_error_bar(data_edit[data_edit.columns.tolist()])
    fig = go.Figure()
    fig.add_trace(go.Bar(
    x=data_edit.columns.tolist(), y=data_edit[data_edit.columns.tolist()].iloc[i],
    error_y=dict(type='data', array=error_bar)
    ))
    return fig


@app.callback(
    Output('line-plot', 'figure'),
    Input('Sample_position', 'value'))
def make_line_plot(position_list):
    import plotly.express as px
    fig = px.line(data, x="Time", y=position_list)
    return fig


@app.callback(
    Output(component_id='my-output', component_property='children'),
    Input(component_id='my-input', component_property='value'))
def recommand_time(point):
    value_list = data[point].values.tolist()
    
    for i in range(1, len(value_list)):
        if value_list[i] - value_list[i-1] <= 0:
            time = data["Time"][int((i)/2)]
            return f"The recommanded Time is {time} at {point}"    
    return "It keeps increasing!"


if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
